## Importing relevant libraries


In [1]:
import nltk
import re
import pdfplumber
import pytesseract
from pdf2image import convert_from_path
import pandas as pd
import json
import pymongo
import xml.etree.cElementTree as et
import os
import spacy
from spacy import displacy

## Setting environment variables

In [2]:
poppler_path=r'C:\Users\H501271\Downloads\Release-22.04.0-0\poppler-22.04.0\Library\bin'
pytesseract.pytesseract.tesseract_cmd = r"C:\Users\H501271\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"

In [5]:
input_path=input('Enter the path to your folder:')
#Model-1 :C:\Users\H501271\Desktop\IB\ucs_poc\InstructorBot\get_knowledge_base\info_files\details
#Model-2 :C:\Users\H501271\Desktop\Model2_info

Enter the path to your folder:C:\Users\H501271\Desktop\Model2_info


## Converting pdf to image

In [6]:
file_names=os.listdir(input_path)
images=[]
for file in file_names:
    images.extend(convert_from_path(input_path+'/'+file, poppler_path=poppler_path))

In [7]:
pages=[]
for f in file_names:
    pdf=pdfplumber.open(input_path+'/'+f)
    pages.extend(pdf.pages)

## Collecting snaps of only text

In [8]:
all_images_wt=[]
i=0

for page in pages:
    img=images[i]
    bboxes = [
            table.bbox
            for table in page.find_tables()
        ]
    h_page=page.height
    word_top=page.extract_words()[0]['top']
    ratio=(img.size[1]*1.0)/h_page
    crop_left=0
    crop_right=page.width*ratio
    crop_top=word_top*ratio
    crop_bottom=h_page*ratio
    any_table=0
    n_tables=len(bboxes)
    n=1
    for bbox in bboxes:
            any_table=1
            x0, top, x1, bottom = bbox
            crop_bottom=top*ratio
            if crop_top<crop_bottom:
                all_images_wt.append(img.crop((crop_left, crop_top, crop_right, crop_bottom)))
            crop_top=bottom*ratio
            if(n==n_tables):
                all_images_wt.append(img.crop((crop_left, crop_top, crop_right, h_page*ratio))) 
            n=n+1
    if(any_table==0):
        all_images_wt.append(img)
    i=i+1       
      


## Extracting text from the snaps

In [9]:
all_text=''
for img in all_images_wt:
    ocr_text = pytesseract.image_to_string(img)
    all_text=all_text+ocr_text


## Denoising extracted text

In [10]:
# all_text=all_text.split()
lines=re.split('\.|\.\n\n|\n\n',all_text)
new_text=[]
for line in lines:
#     print(line)
#     print("-----")
    line = line.replace('\n',' ')
    line=re.sub('\s+',' ',line)
    if len(line.split(' ')) > 5 and "following" not in line:
        new_text.append(line)


## Get Equipment and tag list

In [11]:
def get_list(file):
    f=open(file)
    lines = f.readlines()
    tags=[]
    for line in lines:
        if line.startswith('#'):
            continue
        else:
            tags.append(line.split('.')[0])
    df=pd.DataFrame(tags,columns=["Tag"]) 
    df=df.drop_duplicates(keep='first')
    df=df.reset_index(drop=True)
    return df
    

In [15]:
def get_list_d(file):
    tag=[]
    description=[]
    reader=pdfplumber.open(file)
    for num_page in range(len(reader.pages)):
        tables=reader.pages[num_page].extract_tables(table_settings={"vertical_strategy": "lines", 
        "horizontal_strategy": "lines"})
        for table in tables:
            n_table=[]
            for row in table:
                while "" in row:
                    row.remove("")
                while None in row:
                    row.remove(None)
                n_table.append(row)

            if "Tag" in n_table[0] and "Description" in n_table[0]:
                di=n_table[0].index("Description")
                ti=n_table[0].index("Tag")
                for row in n_table[1:]:
                    tag.append(row[ti])
                    description.append(row[di])

    df=pd.DataFrame(list(zip(tag, description)),columns=['Tag','Description'])
    df=df.drop_duplicates(keep='first')
    df=df.reset_index(drop=True)
    
    return df


In [13]:
tag_file=input("Enter the path to tag file : ")
##"C:\Users\H501271\Desktop\Project\Advanced Distillation\Advanced Distillation\Advanced Distillation Simulator\USD\usdTags.dat"
##"C:\Users\H501271\Downloads\Basic Operations\Basic Operations\Basic Operations Simulator\Process\usdTags.dat"

Enter the path to tag file : C:\Users\H501271\Downloads\Basic Operations\Basic Operations\Basic Operations CourseContent\content\2 Instruments Cond Equip.pdf


In [16]:
# df1=get_list_d("./info_files/W2 - ADVD - Equipment List.pdf")
# df2=get_list_d("./info_files/W2 - ADVD - Steady State Conditions.pdf")
df=get_list_d(tag_file)

In [17]:
nlp = spacy.load("en_core_web_sm")

In [18]:
des=[]
for tag in df["Tag"]:
    description=''
    for para in new_text :
        if tag in para:
            description=description+'.'+para
        elif tag[2:] in para:
            description=description+'.'+para
#         elif str(df2[(df2["Tag"]==tag)]["Description"]) in para:
#             description=description+'.'+para
    sent=description.split('.')
    tag_info=[]
    for s in sent:
        if s=='':
            continue
        tag_info.append(s.strip())

    para='. '.join(tag_info)
    re_inf=[]
    for s in para.split('.'):
        doc=nlp(s.strip())
        for token in doc:

            if (token.text==tag and ((token.dep_=="nsubj" and (token.head.tag_== "VBZ" or token.head.tag_ == "VBN" or token.head.tag_ == "VBP"))or (token.dep_=='appos' and (token.head.head.tag_ == "VBZ" or token.head.head.tag_ == "VBN" or token.head.head.tag_ == "VBP")))):
                re_inf.append(s.strip())
    des.append('.'.join(re_inf))

df["Add_info"]=des           

In [19]:
df.head(10)

,Tag,Description,Add_info
0,FV322,Flow Control Valve (Feed),The flow control valve (FV322) is a variable c...
1,HV312,Hand Valve (Feed),The hand valve (HV312) is a variable control v...
2,HV331,Hand Valve (Outlet),The hand valve (HV331) is a variable control v...
3,SV302,Solenoid Valve (Feed),The solenoid valve (SV302) is a two position v...
4,TK330,Tank,The tank (TK330) is a simple storage tank that...
5,Range,Units,
6,01FC322,Feed Flow,The PID controller (01FC322) displays the flow...
7,01FI302,Feed Flow,
8,01FI312,Feed Flow,
9,01FI332,Outlet Flow,


In [16]:
path_kpv=input("Enter the path to KPV files : ")

Enter the path to KPV files : ./info_files/xml/KPVs


In [17]:
roots_kpv=[]
files_kpv=os.listdir(path_kpv)
for file in files_kpv:
    tree=et.parse(path_kpv+'/'+file)
    roots_kpv.append(tree.getroot())

In [18]:
lower_limit=[]
high_limit=[]
tag_name=[]
unit=[]
for i in range(0,8):
    root=roots_kpv[i]

    for elem in root.findall('Items/EvaluationItemBase/LowLimitsTrend/XmlRepresentation'):
          lower_limit.append(elem.text.split(',')[-1].split(':')[-1])
          tag_name.append(elem.text.split(',')[0].split('.')[0])
          unit.append(elem.text.split(',')[1])

    for elem in root.findall('Items/EvaluationItemBase/HighLimitsTrend/XmlRepresentation'):
          high_limit.append(elem.text.split(',')[-1].split(':')[-1])


df = pd.DataFrame(list(zip(tag_name, lower_limit,high_limit,unit)), columns=["TagName", "LowLimits", "HighLimits", "Unit"])
df = df.drop_duplicates(subset=['TagName'])
df=df.reset_index(drop=True)
df.head(10)

,TagName,LowLimits,HighLimits,Unit
0,11FC20,19.000,21.000,MBPD
1,11FC23,19.000,22.000,MBPSD
2,11FC24,8.000,12.000,MBPSD
3,11PC16,165.000,175.000,PSIG
4,11PC19,260.000,270.000,PSIG
5,11PI17,275.000,282.000,PSIG
6,11TI39,120.000,128.000,DEG F
7,11FC15,85.000,135.000,MLB/HR
8,11FC17,35.600,58.000,MBPSD
9,11FC19,24.000,40.000,MBPSD


In [19]:
df_new=pd.DataFrame(columns=['Tag','Info','LowLimit','HighLimit'])
for index,row in df2.iterrows():
    ll=''
    hl=''
    tag=row["Tag"]
    if row["Add_info"]=="":
        info=row["Description"]
    else:
        info=row["Add_info"]
    arr=df[df['TagName'] == tag].index
    
    if(len(arr)):
      i=arr[0]
      ll=df.iloc[i]["LowLimits"]
      hl=df.iloc[i]["HighLimits"]
    row = {'Tag': tag,'Info':info,'LowLimit':ll,'HighLimit':hl}
    df_new = df_new.append(row, ignore_index = True)
        
df_new.head(10)

,Tag,Info,LowLimit,HighLimit
0,11AC12,11AC12 regulates the flow of reflux from drum ...,0.900,1.100
1,11AC17,The Depropanizer analyzer controller 11AC17 is...,0.800,1.200
2,11AI16A,T-200 Bottoms Analyzer,,
3,11AI16B,T-200 Bottoms Analyzer,,
4,11ESD11,E-100 Steam Slam Trip,,
5,11ESD15,E-220 Steam Slam Trip,,
6,11FC01,Heavy Feed to Debutanizer,40.000,55.000
7,11FC02,Light Feed to Debutanizer,,
8,11FC15,E-100 Steam Flow,85.000,135.000
9,11FC17,T-100 Bottom to Storage Tank 1,35.600,58.000


In [20]:
result=df.to_json(orient="records")
parsed = json.loads(result)
print(json.dumps(parsed, indent=5) )

[
     {
          "Tag": "FV322",
          "Description": "Flow Control Valve (Feed)",
          "Add_info": "The flow control valve (FV322) is a variable control valve"
     },
     {
          "Tag": "HV312",
          "Description": "Hand Valve (Feed)",
          "Add_info": "The hand valve (HV312) is a variable control valve"
     },
     {
          "Tag": "HV331",
          "Description": "Hand Valve (Outlet)",
          "Add_info": "The hand valve (HV331) is a variable control valve that controls the flow of water out of the tank"
     },
     {
          "Tag": "SV302",
          "Description": "Solenoid Valve (Feed)",
          "Add_info": "The solenoid valve (SV302) is a two position valve that is opened and closed by a hand switch (01HS302)"
     },
     {
          "Tag": "TK330",
          "Description": "Tank",
          "Add_info": "The tank (TK330) is a simple storage tank that is open to the atmosphere"
     },
     {
          "Tag": "Range",
          "Description"

In [20]:
result=df_new.to_json(orient="records")
parsed = json.loads(result)
print(json.dumps(parsed, indent=5) )

[
     {
          "Tag": "11AC12",
          "Info": "11AC12 regulates the flow of reflux from drum D-100 to the Debutanizer column, in order to control the overhead composition to 1 mol % of Cs.11AC12 has an analysis time of 300 seconds",
          "LowLimit": "0.900",
          "HighLimit": "1.100"
     },
     {
          "Tag": "11AC17",
          "Info": "The Depropanizer analyzer controller 11AC17 is designed to control the composition of the overheads flow to 1 mol % C4 by cascading to flow controller 11FC23.11AC17 has an analysis time of 150 seconds",
          "LowLimit": "0.800",
          "HighLimit": "1.200"
     },
     {
          "Tag": "11AI16A",
          "Info": "T-200 Bottoms Analyzer",
          "LowLimit": "",
          "HighLimit": ""
     },
     {
          "Tag": "11AI16B",
          "Info": "T-200 Bottoms Analyzer",
          "LowLimit": "",
          "HighLimit": ""
     },
     {
          "Tag": "11ESD11",
          "Info": "E-100 Steam Slam Trip",
       

In [21]:
result=df1.to_json(orient="records")
parsed = json.loads(result)
print(json.dumps(parsed, indent=5) )

[
     {
          "Tag": "D-100",
          "Description": "Debutanizer Reflux Drum"
     },
     {
          "Tag": "D-200",
          "Description": "Depropanizer Reflux Drum"
     },
     {
          "Tag": "E-100",
          "Description": "Debutanizer Reboiler"
     },
     {
          "Tag": "E-110",
          "Description": "Debutanizer Condenser"
     },
     {
          "Tag": "E-210",
          "Description": "Depropanizer Condenser"
     },
     {
          "Tag": "E-220",
          "Description": "Depropanizer Reboiler"
     },
     {
          "Tag": "E-230",
          "Description": "C3 Product Cooler"
     },
     {
          "Tag": "E-240",
          "Description": "C4 Product Cooler"
     },
     {
          "Tag": "E-250",
          "Description": "Depropanizer Feed/Bottoms Exchanger"
     },
     {
          "Tag": "P-62",
          "Description": "Reflux pump (from D-100 to T-100)"
     },
     {
          "Tag": "P-63",
          "Description": "Spare reflux pump 

In [22]:
df_new.to_json('tag_info.json',orient="records")
df1.to_json('equipment_info.json',orient="records")

In [23]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")

In [24]:
database=myclient.knowledge_base
database.equipment_info.drop()
database.tag_info.drop()
collection_1=database.equipment_info
collection_2=database.tag_info

In [25]:
with open('tag_info.json') as file:
    data_tag = json.load(file)

In [26]:
with open('equipment_info.json') as file:
    data_equip = json.load(file)

In [27]:
collection_1.insert_many(data_equip)
collection_2.insert_many(data_tag)